In [ ]:
!pip install ../input/sklearn24/scikit_learn-0.24.0-cp37-cp37m-manylinux2010_x86_64.whl
!pip freeze | grep scikit-learn

# Neural Network Starter Pytorch Version


In [ ]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings('ignore')

import os, gc, random, time
import datatable as dt
import pickle
# import cudf
import pandas as pd
import numpy as np

# import cupy as cp
import janestreet
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve, log_loss
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

#!pip uninstall scikit-learn
#!pip install scikit-learn==0.24.1
print('OK')

In [ ]:
def seed_everything(seed=2021):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
#seed_everything(seed=2020)
Neutralize = False

# Loading

In [ ]:
%%time
print('Loading...')
train_datatable = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')# 用datatable会快一点
train = train_datatable.to_pandas()
del train_datatable

train.drop(train[(train['date']<85)].index,inplace=True)
train.drop(train[(train['date']==294)].index,inplace=True)

print("Len: ",len(train))

# Feature engineering

In [ ]:
features_fill = [c for c in train.columns if 'feature' in c]

print('Filling...')
f_mean = train[features_fill[1:]].mean()
train = train.loc[train.weight > 0].reset_index(drop = True)
train[features_fill[1:]] = train[features_fill[1:]].fillna(f_mean)

train['cross_41_42_43'] = train['feature_41'] * train['feature_42'] * train['feature_43']
train['cross_1_2'] = train['feature_1'] / (train['feature_2'] + 1e-5 )

train['weight_x_resp'] =  train['weight']*train['resp']
train['weight_x_resp1'] = train['weight']*train['resp_1']
train['weight_x_resp2'] = train['weight']*train['resp_2']
train['weight_x_resp3'] = train['weight']*train['resp_3']
train['weight_x_resp4'] = train['weight']*train['resp_4']
train['bias'] = 1
bins = 100
train['daily_wr_bin'] = (
    train.groupby('date')['weight_x_resp']
              .apply(pd.qcut, q=bins, labels=False, duplicates='drop')
)

train['daily_wr3_bin'] = (
    train.groupby('date')['weight_x_resp3']
              .apply(pd.qcut, q=bins, labels=False, duplicates='drop')
)
train['daily_wr4_bin'] = (
    train.groupby('date')['weight_x_resp4']
              .apply(pd.qcut, q=bins, labels=False, duplicates='drop')
)

train['daily_wr1_bin'] = (
    train.groupby('date')['weight_x_resp1']
              .apply(pd.qcut, q=bins, labels=False, duplicates='drop')
)
train['daily_wr2_bin'] = (
    train.groupby('date')['weight_x_resp2']
              .apply(pd.qcut, q=bins, labels=False, duplicates='drop')
)


train['action'] =  (train['daily_wr_bin'] > bins//2).astype(int)
train['action1'] = (train['daily_wr1_bin'] > bins//2).astype(int)
train['action2'] = (train['daily_wr2_bin'] > bins//2).astype(int)
train['action3'] = (train['daily_wr3_bin'] > bins//2).astype(int)
train['action4'] = (train['daily_wr4_bin'] > bins//2).astype(int)

#train['action'] = (train['resp'] > 0).astype('int')
#train['action_1'] = (train['resp_1'] > 0).astype('int')
#train['action_2'] = (train['resp_2'] > 0).astype('int')
#train['action_3'] = (train['resp_3'] > 0).astype('int')
#train['action_4'] = (train['resp_4'] > 0).astype('int')

target_cols = ['action', 'action4']#'action_1', 'action_2']
print('Finish.')

# Reduce memory

In [ ]:
def reduce_memory_usage(df):   
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                    
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df
print("Reducing Memory")

# Build_neutralizer

In [ ]:
def build_neutralizer(train, features, proportion, return_neut=False):
    """
    Builds neutralzied features, then trains a linear model to predict neutralized features from original
    features and return the coeffs of that model.
    """
    neutralizer = {}
    neutralized_features = np.zeros((train.shape[0], len(features)))
    target = train[['resp', 'bias']].values
    for i, f in enumerate(features):
        # obtain corrected feature
        feature = train[f].values.reshape(-1, 1)
        coeffs = np.linalg.lstsq(target, feature)[0]
        neutralized_features[:, i] = (feature - (proportion * target.dot(coeffs))).squeeze()
        
    # train model to predict corrected features
    neutralizer = np.linalg.lstsq(train[features+['bias']].values, neutralized_features)[0]
    
    if return_neut:
        return neutralized_features, neutralizer
    else:
        return neutralizer

def neutralize_array(array, neutralizer):
    neutralized_array = array.dot(neutralizer)
    return neutralized_array


def test_neutralization():
    dummy_train = train.loc[:100000, :]
    proportion = 1.0
    neutralized_features, neutralizer = build_neutralizer(dummy_train, features, proportion, True)
    dummy_neut_train = neutralize_array(dummy_train[features+['bias']].values, neutralizer)
    
#     assert np.array_equal(neutralized_features, dummy_neut_train)
    print(neutralized_features[0, :10], dummy_neut_train[0, :10])

if Neutralize:
    proportion = 1.0
    neutralizer = build_neutralizer(train, features, proportion)
    train[features] = neutralize_array(train[features+['bias']].values, neutralizer)
train = reduce_memory_usage(train)

In [ ]:
features=['feature_89',
 'feature_67',
 'feature_115',
 'feature_72',
 'feature_44',
 'feature_91',
 'feature_79',
 'feature_78',
 'feature_80',
 'feature_3',
 'feature_37',
 'feature_86',
 'feature_77',
 'feature_19',
 'feature_60',
 'feature_9',
 'feature_50',
 'feature_0',
 'feature_51',
 'feature_29',
 'feature_73',
 'feature_90',
 'feature_70',
 'feature_17',
 'feature_31',
 'feature_109',
 'feature_85',
 'feature_121',
 'feature_62',
 'feature_39',
 'feature_95',
 'feature_46',
 'feature_56',
 'feature_7',
 'feature_87',
 'feature_42',
 'feature_92',
 'feature_103',
 'feature_53',
 'feature_21',
 'feature_43',
 'feature_104',
 'feature_75',
 'feature_81',
 'feature_52',
 'feature_55',
 'feature_64',
 'feature_27',
 'feature_13',
 'feature_1',
 'feature_98',
 'feature_84',
 'feature_65',
 'feature_54',
 'feature_5',
 'feature_45',
 'feature_41',
 'feature_11',
 'feature_74',
  'cross_41_42_43',
  'cross_1_2']

# Training


In [ ]:
dropout_rates = [0.10143786981358652, 0.19720339053599725, 0.2703017847244654, 0.23148340929571917, 0.2357768967777311]
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(features))
        self.dropout0 = nn.Dropout(dropout_rates[0])

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(features), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rates[1])

        self.dense2 = nn.Linear(hidden_size+len(features), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rates[2])

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rates[3])

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rates[4])

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0

    for data in dataloader:
        optimizer.zero_grad()
        features = data['features'].to(device)
        label = data['label'].to(device)
        outputs = model(features)
        #print('OUTPUTS shape: ',outputs.shape,type(outputs))
        #print('Label shape: ',label.shape,type(label))
        loss = loss_fn(outputs, label)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

        final_loss += loss.item()

    final_loss /= len(dataloader)

    return final_loss

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []

    for data in dataloader:
        features = data['features'].to(device)
        with torch.no_grad():
            outputs = model(features)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds)

    return torch.tensor(preds,dtype=torch.float64)

In [ ]:
class MarketDataset:
    def __init__(self, df,features,labels):
        self.features = df[features].values
        self.label = df[labels].values

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.features[idx].astype(np.float64), dtype=torch.float),
            'label': torch.tensor(self.label[idx].astype(np.float64), dtype=torch.float)
        }

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss
    
# Ealrly stop stratagem    
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
 
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            self.save_checkpoint(epoch_score, model, model_path)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score
        
        
def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
%%time

from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5,max_train_size=1241711)
print(tscv)
print(len(train))
#gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
#splits = list(gkf.split(y, groups=train['date'].values))
for fold, (train_indices, test_indices) in enumerate(tscv.split(train)):
    print('Fold: ',fold)
    print('train_indices:{} Len:{}'.format(train_indices,len(train_indices)))
    print('test_indices:{}  Len:{}'.format(test_indices,len(test_indices)))

In [ ]:
batch_size = 4096
label_smoothing = 1e-3
learning_rate = 1e-3
epochs = 50
seed_everything(seed=2020)
#feat_cols = [f'feature_{i}' for i in range(130)]

from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5,max_train_size=1241711)
start_time = time.time()
oof = np.zeros(len(train['action']))

for fold, (tr, te) in enumerate(tscv.split(train)):
    train_set = MarketDataset(train.loc[tr],features,target_cols)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_set = MarketDataset(train.loc[te],features,target_cols)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=4)
    
    torch.cuda.empty_cache()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Model()
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = SmoothBCEwLogits(smoothing=label_smoothing)
    
    ckp_path = f'JSModel_{fold}.pth'
    es = EarlyStopping(patience=epochs/5, mode="max")
    for epoch in range(epochs):
        train_loss = train_fn(model, optimizer, None, loss_fn, train_loader, device)
        valid_pred = inference_fn(model, valid_loader, device)
        auc_score = 0
        for i in range(len(target_cols)):
            auc_score+=roc_auc_score((train.loc[te][target_cols[i]] > 0).astype('int').values.reshape(-1, 1), valid_pred[:,i])/len(target_cols)
        #logloss_score = log_loss((train.loc[te]['resp'] > 0).astype('int').values.reshape(-1, 1), valid_pred)
        logloss_score = loss_fn(valid_pred,torch.tensor(np.array(train.loc[te][target_cols].values),dtype=torch.float64))
        #valid_pred = np.where(valid_pred >= 0.5, 1, 0).astype(int)
        u_score = utility_score_bincount(date=train.loc[te].date.values, weight=train.loc[te].weight.values, 
                                         resp=train.loc[te].resp.values, action=np.where(valid_pred.mean(axis=1) >= 0.5, 1, 0).astype(int))

        print(f"FOLD{fold} EPOCH:{epoch:3}, train_loss:{train_loss:.5f},auc:{auc_score:.5f},u_score:{u_score:.5f}, "
              f"time: {(time.time() - start_time) / 60:.2f}min")
        
        es(u_score, model, model_path=ckp_path)
        if es.early_stop:
            print("Early stopping")
            break
#     break # only train 1 model for fast, you can remove it to train 5 folds

# Tensorflow Part

In [ ]:
class EarlyStopping_Tensorflow:
    def __init__(self, patience=7, mode="max", delta=0.):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
 
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            #print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            # ema.apply_shadow()
            self.save_checkpoint(epoch_score, model, model_path)
            # ema.restore()
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            clf.save('{}'.format(model_path))
        self.val_score = epoch_score

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices


SEED = 2020

np.random.seed(SEED)

X_train = train[features]
#y_train = (train.loc[:, 'action'])

y_train = train[target_cols]

# fit
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

epochs = 200
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5,max_train_size=1241711)
start_time = time.time()
for fold, (tr, te) in enumerate(tscv.split(train)):
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(SEED)
    clf = create_mlp(
    len(features), len(target_cols), hidden_units, dropout_rates, label_smoothing, learning_rate
    )
    
    X_train = train[features].loc[tr]
    X_valid = train[features].loc[te]
#y_train = (train.loc[:, 'action'])
    y_train = train[target_cols].loc[tr]
    y_valid = train[target_cols].loc[te]
    
    ckp_path = f'TensorModel_{fold}.h5'
    es = EarlyStopping_Tensorflow(patience=5, mode="max")

    for i in range(50):
        clf.fit(X_train, y_train, epochs=3, batch_size=batch_size, verbose=2)
        y_pred = clf.predict(X_valid)
        
        u_score = utility_score_bincount(date=train.loc[te].date.values, weight=train.loc[te].weight.values, 
                                         resp=train.loc[te].resp.values, action=np.where(y_pred.mean(axis=1) >= 0.5, 1, 0).astype(int))
        es(u_score, clf, model_path=ckp_path)
        if es.early_stop:
            print("Early stopping")
            break

# Just upvoting if it helps!!!

In [ ]:
for _fold in range(5):
        torch.cuda.empty_cache()
        model = Model()
        model.to(device)
        model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
        model.load_state_dict(torch.load(model_weights))
        model.eval()
        model_list.append(model)
model.load_state_dict(
)